# Quora Insincere Questions Classification

## By Koby Nimni, Noam Frank, and Ziv Bar Nahum

## Project Outline:
#### 1. Import Libraries and Data
#### 2. Raw Data “Descriptive Statistics” 
#### 3. Cleaning the Data and Pre-Processing
#### 4.	Visualization of the Clean Data
#### 5.	Vectorising the Data
#### 6.	Under Sampling and clustering
#### 7.	Modelling and Results
#### 8.	Concluding Remarks and Work in Progress 

## 1. Import Libraries and Data

### 1.1 Import  DS Basics and NLP Specifics:

In [ ]:
################# GENERAL IMPORTS
import os
import string
from pprint import pprint
from operator import itemgetter
import matplotlib.pyplot as plt
from sklearn.svm import LinearSVC
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import f1_score, precision_score, recall_score,accuracy_score
##################################### NLP SPECIFIC IMPORTS
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk import ngrams
from nltk.stem import PorterStemmer
from nltk.corpus import reuters
from sklearn.feature_extraction.text import CountVectorizer
from wordcloud import WordCloud
from collections import Counter
reuters.fileids()
stopwords.words('english')

import numpy as np
import pandas as pd
import warnings
from sys import modules

warnings.filterwarnings('ignore')
%matplotlib inline

from gensim.models import word2vec
import logging

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten
from keras.layers.wrappers import TimeDistributed
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM
from keras.layers import Dropout

import seaborn as sns
sns.set(style = 'darkgrid')
print(os.listdir("../input"))
import re
pd.set_option('max_colwidth', 800)
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

np.random.seed(1234)
%matplotlib inline

print('all set')

### 1.2 Load the Text Data

In [ ]:
quora_train=pd.read_csv("../input/train.csv")
quora_test=pd.read_csv("../input/test.csv")
print("Train size =" ,quora_train.shape)
print("Test size =" ,quora_test.shape)

In [ ]:
# quora_train=quora_train[0:5000]
# quora_test=quora_test[0:1000]

In [ ]:
quora_train.head()

## 2.	Raw Data “Descriptive Statistics”

In [ ]:
quora_train.info()

* #### Dataset has 3 columns and 1,306,122 questions.
* #### No missing data.

> ### Sincere question examples: 

In [ ]:
sincere = quora_train[quora_train.target==0]
insincere = quora_train[quora_train.target==1]

In [ ]:
[print(q,'\n') for q in sincere['question_text'][[1,6,40,300,120]]]

### 2.1 Target Distribution

In [ ]:
quora_train['target'].value_counts().plot(kind='bar', title='Target distribution')

In [ ]:
round(quora_train['target'].value_counts(normalize =True),3)*100

### => Data is Imbalanced ! 
### Target is only 6.2% of the data

### 2.2 Word and Character Count 

In [ ]:
quora_train['words'] = quora_train.question_text.apply(lambda x: len(x.split()))
quora_train['characters'] = quora_train.question_text.apply(lambda x: len(x))
quora_test['words'] = quora_test.question_text.apply(lambda x: len(x.split()))
quora_test['characters'] = quora_test.question_text.apply(lambda x: len(x))

In [ ]:
quora_train.head()

### 2.2.1 Average Word and Character Count by Target

In [ ]:
fig = plt.figure(figsize=(18, 7))

plt.subplot(1, 2, 1)
quora_train.groupby('target')['words'].mean().plot(kind='bar', ylim=(0,20), title= 'Average word count by target')

plt.subplot(1, 2, 2)
quora_train.groupby('target')['characters'].mean().plot(kind='bar', ylim=(0,105), title= 'Average character count by target')

* ### Insincere questions have on average longer word counts and character counts.
* ### We will use these as features in our model.

In [ ]:
fig = plt.figure(figsize=(18, 8))
font = {'size': 16, 'weight': 'bold'}

plt.subplot(2, 1, 1)
plt.title('Word count - by outcome',fontdict=font)
plt.xlim(0,70)
ax = sns.boxplot(x="words", y="target", data=quora_train, orient="h")

plt.subplot(2, 1, 2)
plt.title('character count - by outcome',fontdict=font)
plt.xlim(0,350)
ax = sns.boxplot(x="characters", y="target", data=quora_train, orient="h")

* ### Insincere questions have larger variance and median for both word and character counts.

###  2.3 Additional Features - Verb and Noun Count 

In [ ]:
from nltk import pos_tag

def verb_count(text):
    token_text= word_tokenize(text)
    tagged_text = pos_tag(token_text)
    counter=0
    for w,t in tagged_text:
        t = t[:2]
        if t in ['VB']:
            counter+=1
    return counter

def noun_count(text):
    token_text= word_tokenize(text)
    tagged_text = pos_tag(token_text)
    counter=0
    for w,t in tagged_text:
        t = t[:2]
        if t in ['NN']:
            counter+=1
    return counter


In [ ]:
quora_train.head()

 # 3. Cleaning the Data and Pre-Processing

 ###  3.1 Lower-casing

In [ ]:
quora_train['question_text_prep'] = quora_train['question_text'].apply(lambda x: x.lower())
quora_test['question_text_prep'] = quora_test['question_text'].apply(lambda x: x.lower())

### 3.2 Punctuations and Stop-words

In [ ]:
def pad_punctuation_w_space(string):
    s = re.sub('([:;"*.,!?()/\=-])', r' \1 ', string)
    s=re.sub('[^a-zA-Z]',' ',s)
    s = re.sub('\s{2,}', ' ', s)
    s =  re.sub(r"\b[a-zA-Z]\b", "", s) #code for removing single characters
    return s
quora_train['question_text_prep'] = quora_train['question_text_prep'].apply(lambda x: pad_punctuation_w_space(x))
quora_test['question_text_prep'] = quora_test['question_text_prep'].apply(lambda x: pad_punctuation_w_space(x))


### 3.3	Questions as List of Strings

In [ ]:
quora_train['question_text_prep'] = quora_train['question_text_prep'].apply(lambda x: x.split())
quora_test['question_text_prep'] = quora_test['question_text_prep'].apply(lambda x: x.split())

In [ ]:
stop_list = stopwords.words('english') + list(string.punctuation)
quora_train['question_text_prep'] = quora_train['question_text_prep'].apply(lambda x: [i for i in x if i not in stop_list])
quora_test['question_text_prep'] = quora_test['question_text_prep'].apply(lambda x: [i for i in x if i not in stop_list]) 


#### ** this was done because word embedding needs the text to be represented as list of strings.

## 4. Visualization of the Clean Data

### 4.1	Unigram Distribution – Sincere vs. Insincere Questions

In [ ]:
quora_train['question_text_prep_string'] = quora_train['question_text_prep'].str.join(" ")
quora_test['question_text_prep_string'] = quora_test['question_text_prep'].str.join(" ")

In [ ]:
# quora_insincere =  quora_train[quora_train.target==1]
# quora_sincere =  quora_train[quora_train.target==0]

# # make "all in one" corpuses for the 2 classes in the target
# insincere_all_in_one = ' '.join([q for q in quora_insincere.question_text_prep_string])
# sincere_all_in_one = ' '.join([q for q in quora_sincere.question_text_prep_string]) 

### 4.1.1 Word Cloud for Sincere and Insincere Questions

In [ ]:
# fig = plt.figure(figsize=(30, 12))
# font = {'size': 20, 'weight': 'bold'}

# plt.subplot(1, 2, 1)
# plt.title('Insincere questions',fontdict=font)
# cloud1 = WordCloud(max_words=100,width=480, height=480, background_color='grey')
# cloud1.generate_from_text(insincere_all_in_one)
# plt.imshow(cloud1)
# plt.axis('off')

# plt.subplot(1, 2, 2)
# plt.title('Sincere questions',fontdict=font)
# cloud = WordCloud(max_words=100,width=480, height=480, background_color='skyblue')
# cloud.generate_from_text(sincere_all_in_one)
# plt.imshow(cloud)
# plt.axis('off')

### 4.2	Bigram and Trigram Distribution - Sincere vs. Insincere Questions

In [ ]:
# insincere_tokens = [t for t in word_tokenize(insincere_all_in_one)]
# sincere_tokens = [t for t in word_tokenize(sincere_all_in_one)]

In [ ]:
# from collections import Counter
# from nltk import ngrams
# bi_gram_insincere = Counter(ngrams(insincere_tokens, 2))
# tri_gram_insincere = Counter(ngrams(insincere_tokens, 3))
# bi_gram_sincere = Counter(ngrams(sincere_tokens, 2))
# tri_gram_sincere = Counter(ngrams(sincere_tokens, 3))

In [ ]:
# bi_gram_ins = pd.DataFrame(bi_gram_insincere.most_common(20), columns=['bi_gram_ins','frequency'])
# bi_gram_sin = pd.DataFrame(bi_gram_sincere.most_common(20), columns=['bi_gram_sin','frequency'])
# tri_gram_ins = pd.DataFrame(tri_gram_insincere.most_common(20), columns=['tri_gram_ins','frequency'])
# tri_gram_sin = pd.DataFrame(tri_gram_sincere.most_common(20), columns=['tri_gram_sin','frequency'])

### 4.2.1 Bigram Distribution - Sincere vs. Insincere Questions

In [ ]:
# import matplotlib.pyplot as plt
# fig, (ax, ax2) = plt.subplots(ncols=2, sharex=False)
# fig.subplots_adjust(wspace =0.6)
# ax.invert_xaxis()

# bi_gram_ins.sort_values(by='frequency').plot(kind='barh', x='bi_gram_ins', legend=True, ax=ax, figsize=(18,9), fontsize =16)
# bi_gram_sin.sort_values(by='frequency').plot(kind='barh', x='bi_gram_sin',ax=ax2, figsize=(18,9),fontsize =16)
# plt.show()

* ### The most common bigram among sincere questions is "best way" (e.g., what is the best way...?).
* ### Not surprisingly, "Donald Trump" is the most common bigram among insincere questions (obviously all of these instances are statements/opinions on "Donald Trump").

### 4.2.2 Trigram Distribution - Sincere vs. Insincere Questions

In [ ]:
# fig, (ax, ax2) = plt.subplots(ncols=2, sharex=False)
# fig.subplots_adjust(wspace =0.8)
# ax.invert_xaxis()

# tri_gram_ins.sort_values(by='frequency').plot(kind='barh', x='tri_gram_ins', legend=True, ax=ax, figsize=(18,9), fontsize =16)
# tri_gram_sin.sort_values(by='frequency').plot(kind='barh', x='tri_gram_sin',ax=ax2, figsize=(18,9), fontsize =16)
# plt.show()

* ### The most common trigram among sincere questions is "advice would give" (e.g., what advise would you give for ..?).
* ### "Kim Jong Un" is the most common trigram among insincere questions (obviously all of these instances are statements/opinions on "Kim Jong Un").
* ### It seems that the most common bigrams and trigrams are revolved around different topics and therefore cluster analysis might be beneficial. 

## 5. Vectorising the Data

In [ ]:
sents = list(quora_train.question_text_prep.values) 
sents[0]

### 5.1	Using the "Glove" Word Embedding

In [ ]:
min_num = 3 # minimum number of occurrences in text
EMBEDDING_FILE= "../input/embeddings/glove.840B.300d/glove.840B.300d.txt"

In [ ]:
import numpy as np
def loadGloveModel(gloveFile):
    print ("Loading Glove Model")
    f = open(gloveFile,'r', encoding='utf8')
    model = {}
    for line in f:
        splitLine = line.split(' ')
        word = splitLine[0]
        embedding = np.asarray(splitLine[1:], dtype='float32')
        model[word] = embedding
    print ("Done.",len(model)," words loaded!")
    return model

In [ ]:
word_model= loadGloveModel(EMBEDDING_FILE)   
# print (word_model['hello']) # if we want to see an example for a vector

In [ ]:
print('Loaded %s word vectors.' % len(word_model))

### 5.2	Handling Unknown Words

### 5.2.1 Identify and Count Unknown Words

In [ ]:
unknown_words = []
for question in quora_train.question_text_prep:
    for word in question:
        if word not in word_model:
            unknown_words.append(word)
        else: pass

In [ ]:
len(unknown_words)

In [ ]:
unknown_words[:10]

### 5.2.2 The Frequency of the Unknown Words

In [ ]:
total_term_frequency = Counter(unknown_words)

for word, freq in total_term_frequency.most_common(20):
    print("{}\t{}".format(word, freq))

### 5.3	Vectorising the Data Using the “word model”

In [ ]:
def get_vector(DataFrame):
    vec_X = []
    i = 0
    for item in DataFrame.question_text_prep_string: 
        
        sentence = pad_punctuation_w_space(item)
        s = np.array([])
        s = []
        if len(sentence)==0:
            s = np.array(word_model['UNK'])
            vec_X.append(s) 
            i += 1
        else:
                for word in sentence.split():
                    if len(s) == 0:
                        try:
                            s = np.array(word_model[word])
                        except: 
                            s = np.array(word_model['UNK'])
                    else:
                        try:
                            s += np.array(word_model[word])
                        except: 
                            s += np.array(word_model['UNK'])         
                vec_X.append(s) 
                i += 1

    return vec_X

In [ ]:
vec_X_train=get_vector(quora_train)
vec_X_test=get_vector(quora_test)
quora_train["vector"]=vec_X_train
quora_test["vector"]=vec_X_test

## 6. Under Sampling
Data is very imballanced. Without Under sampling the classifier will see the target as **noise**  

### 6.1	Proceed with the Under Sample Data 

In [ ]:
from imblearn.under_sampling import RandomUnderSampler

### Two general techniques for working with imballanced data:
<br>
<img src="https://cdn-images-1.medium.com/max/800/1*H6XodlitlGDl9YdbwaZLMw.png">

### We chose random undersampling

In [ ]:
X = quora_train[['words','characters','vector']] #,'noun_count'
y = quora_train['target']

In [ ]:
rus = RandomUnderSampler(return_indices=True, ratio = 0.42)
X_rus, y_rus, id_rus = rus.fit_sample(X, y)

print('indexes:', id_rus)
print(len(id_rus))
print(quora_train.target.value_counts())

In [ ]:
quora_undr=quora_train.loc[id_rus]

In [ ]:
quora_undr['target'].value_counts(ascending=True).plot(kind='bar')

In [ ]:
quora_undr['target'].value_counts(normalize=True)

### Important note:  Undersampling was carried out while preserving a bias in the data.

In [ ]:
quora_under_prep = quora_undr

In [ ]:
quora_under_prep["characters"].head()

**### 6.3	Check Zero-Size Vectors

### important notice- cleaning data using stopwords, removing numbers etc. may leave the line\vector empty and cause later errors. that is why we check if there are  lines with  zero length vector

In [ ]:
quora_under_prep['noun_count'] = quora_under_prep.question_text.apply(lambda x: noun_count(x))
quora_test['noun_count'] = quora_test.question_text.apply(lambda x: noun_count(x))

In [ ]:
quora_under_prep['vector_length']= quora_under_prep['vector'].apply(lambda x: len(x))
quora_test['vector_length']= quora_test['vector'].apply(lambda x: len(x))
quora_test['vector_length'].describe()

In [ ]:
quora_best=quora_under_prep

### We believe that the direction of the vector is affected from the variables we have found earlier. For this reason we decided to join all those variables with the word embedding vector and create a new, more reliable vector. from now on this vector will be our point of reference

In [ ]:
import numpy as np
quora_best["joinvector"]=[np.concatenate((np.array([quora_best["characters"].iloc[i]]),quora_best["vector"].iloc[i]), axis=None) for i in range(len(quora_best))]
quora_best["joinvector_2"]=[np.concatenate((np.array([quora_best["words"].iloc[i]]),quora_best["joinvector"].iloc[i]), axis=None) for i in range(len(quora_best))]
quora_best["joinvector_all"]=[np.concatenate((np.array([quora_best["noun_count"].iloc[i]]),quora_best["joinvector_2"].iloc[i]), axis=None) for i in range(len(quora_best))]

## Doing the same for the test data

In [ ]:
quora_test["joinvector"]=[np.concatenate((np.array([quora_test["characters"].iloc[i]]),quora_test["vector"].iloc[i]), axis=None) for i in range(len(quora_test))]
quora_test["joinvector_2"]=[np.concatenate((np.array([quora_test["words"].iloc[i]]),quora_test["joinvector"].iloc[i]), axis=None) for i in range(len(quora_test))]
quora_test["joinvector_all"]=[np.concatenate((np.array([quora_test["noun_count"].iloc[i]]),quora_test["joinvector_2"].iloc[i]), axis=None) for i in range(len(quora_test))]

### 6.5	Clustering the data

#### We believe that certain categories affect the feelings of the users more than others, that is why we want to understand the differenct clusters within the questions

#### we will use our predefined joined vector in order to cluster our data

In [ ]:
# quora_test.head(1)

In [ ]:
X_joinvec=quora_best["joinvector_all"].tolist()

### performing DBSCAN

In [ ]:
# from sklearn.cluster import DBSCAN
# fit_model_joinvec=DBSCAN(eps=0.25,min_samples=10).fit(X_joinvec)
# fitted_model_joinvec=fit_model_joinvec.labels_
# quora_best['DBSCAN_Cluster_joinvec']=fitted_model_joinvec

# quora_best.head()

In [ ]:
#quora_best['DBSCAN_Cluster_joinvec'].value_counts()

* #### running the code above we saw that DBSCAN cluster did not cluster efficiently

#### let's try KMEANS

In [ ]:
# from sklearn.cluster import KMeans, MiniBatchKMeans
# def calc_inertia(k):
#         model_kmeans= KMeans(n_clusters=k, init='k-means++',verbose=1,random_state=42).fit(X_joinvec)
#         return model_kmeans.inertia_,model_kmeans.labels_

# inertias,labels_kmeans = [(k, calc_inertia(k)) for k in range(1, 21)]

#### Trying KMEAN both with gererating dynamic n_clusters and using a fixed number returned a memory issue that is why we decided to work with a ligther version of KMEANS- called MiniBatchKMeans

In [ ]:
# from sklearn.cluster import KMeans, MiniBatchKMeans
# n_clusters=50
# model_MiniBatch = MiniBatchKMeans(n_clusters=n_clusters, init='k-means++', n_init=1,
#                          init_size=500,
#                          batch_size=500, verbose=1)

# print ("Clustering sparse data with %s" % model_MiniBatch)
# model_MiniBatch.fit(X_joinvec)
# labels_MiniBatch = model_MiniBatch.labels_
# print("done")
# # print("labels", labels_MiniBatch)
# # print("intertia:", model_MiniBatch.inertia_)

In [ ]:
# quora_best['MiniBatch_Cluster_joinvec']=labels_MiniBatch
# # quora_best.head()

In [ ]:
# quora_best['MiniBatch_Cluster_joinvec'].value_counts().head(10)

In [ ]:
# import matplotlib.pylab as plt
# prob_groups = quora_best.groupby("MiniBatch_Cluster_joinvec").MiniBatch_Cluster_joinvec.count()
# prob_groups.plot(kind='hist')
# plt.figure(figsize=(30,100))
# plt.show()

### let's inspect some clusters that have both 1 and 0 values

In [ ]:
# pd.set_option('max_colwidth', 800)
# cluster_columns=quora_best[["question_text","target","MiniBatch_Cluster_joinvec"]]
# random_cluster=cluster_columns[cluster_columns["MiniBatch_Cluster_joinvec"]==38].sort_values(by='target',ascending=False)
# random_cluster.head(10)

In [ ]:
# random_cluster.target.value_counts(normalize=True).plot(kind='bar', title='target', figsize=(10,5))
# plt.show()

#### We also tried  clustering based on tf-idf which did not perform that well

### One option for future processing would be generating a combined vector which includes the cluster dummies

## 7. Modelling and Results 

## 7.1 Train test split

In [ ]:
Features = quora_best['joinvector_all']
# Features2=quora_best['vector']

In [ ]:
# FF=Features.tolist()

In [ ]:
# Features.shape

In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV


X_train, X_val, y_train, y_val = train_test_split(Features,quora_best['target'],
                                                    train_size=0.7, random_state = 143, stratify=quora_best['target'])

## 7.2 Base line classifier

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier

#evaluators:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, f1_score
from sklearn.model_selection import cross_val_score

In [ ]:
# # Lr_clf = LogisticRegression()
# # X =  X_train.tolist()
# # y = y_train

# # Lr_clf.fit(X,y)
# GB_clf=GradientBoostingClassifier()
# X =  X_train.tolist()
# y = y_train

# GB_clf.fit(X,y)

In [ ]:
# f1 = cross_val_score(GB_clf, X, y, scoring='f1', cv=5)
# accuracy = cross_val_score(GB_clf, X, y, cv=5, scoring='accuracy')

In [ ]:
# print('f1 score:{}\nacurracy: {}'.format(round(f1.mean(),2),round(accuracy.mean(),2)))

## 7.3  GridSearchCV for logistic regression classifier

In [ ]:
#weights = [2,3]
# param_grid = {'C': [0.5,10],
#               'class_weight':[{0:1, 1:w} for w in weights]}
# param_grid ={"learning_rate":(0.1,0.5),
#                             'max_depth' : range(2,5,10),
#                             'min_samples_split': range(2,5,10),
#                             'min_samples_leaf' : range(2,5,10),
#                             #'max_features':range(4,8),
#                             "subsample":(0.5,0.8)} 
# param_grid ={"learning_rate":(0.1,0.5),
#                             'max_depth' : range(2,5),
#                             'min_samples_split': range(2,5),
#                             'min_samples_leaf' : range(2,5),
#                             #'max_features':range(4,8),
#                             "subsample":(0.5,0.8)} 

In [ ]:
X_grid5 = X_train.tolist()
y_grid5 = y_train

In [ ]:
#Lr_clf = LogisticRegression()
GB_clf=GradientBoostingClassifier(learning_rate=0.5,max_depth=2,min_samples_split=2,min_samples_leaf=2,subsample=0.8,random_state=0)
GB_clf.fit(X_grid5, y_grid5)
# gs= GridSearchCV(estimator=Lr_clf, param_grid=param_grid, cv=2,scoring='f1') # verbose=15, n_jobs=-1
#gs= GridSearchCV(estimator=GB_clf, param_grid=param_grid, cv=2,scoring='f1') # verbose=15, n_jobs=-1
#gs.fit(X_grid5, y_grid5)
#best_model=gs.best_estimator_ 

In [ ]:
# best_model

### 7.4 Results After Grid Search

In [ ]:
#y_pred= best_model.predict(X_grid5)
y_pred= GB_clf.predict(X_grid5)

In [ ]:
# f1 = cross_val_score(best_model, X_grid5, y_grid5, scoring='f1', cv=2)
# accuracy = cross_val_score(best_model, X_grid5, y_grid5, cv=2, scoring='accuracy')
# cm = confusion_matrix(y_true=y_grid5, y_pred=y_pred)

# print('f1-score:',round(f1.mean(),2),'\naccuracy:',round(accuracy.mean(),2),'\n______________\n')
# cm = cm
# print (classification_report(y_grid5, y_pred))

# print('confusion matrix:')
# pd.DataFrame(cm, 
#              index=best_model.classes_, 
#              columns=best_model.classes_)

f1 = cross_val_score(GB_clf, X_grid5, y_grid5, scoring='f1', cv=3)
accuracy = cross_val_score(GB_clf, X_grid5, y_grid5, cv=3, scoring='accuracy')
cm = confusion_matrix(y_true=y_grid5, y_pred=y_pred)

print('f1-score:',round(f1.mean(),2),'\naccuracy:',round(accuracy.mean(),2),'\n______________\n')
cm = cm
print (classification_report(y_grid5, y_pred))

print('confusion matrix:')
pd.DataFrame(cm, 
             index=GB_clf.classes_, 
             columns=GB_clf.classes_)

## 7.5 Evaluate the model

In [ ]:
X1_val=X_val.tolist()
y1_val=y_val

In [ ]:
# best_model.fit(X1_val,y1_val)
GB_clf.fit(X1_val,y1_val)

In [ ]:
#f1 = cross_val_score(best_model, X1_val,y1_val, scoring='f1', cv=3)
f1 = cross_val_score(GB_clf, X1_val,y1_val, scoring='f1', cv=3)
print('f1 score of "val":', round(f1.mean(),2))

# Using NNclalification model

In [ ]:
# X_NN =  X_train.tolist()
# y_NN = y_train


In [ ]:
# from keras.preprocessing import sequence
# from keras.models import Sequential
# from keras.layers import Dense, Embedding
# from keras.layers import LSTM

In [ ]:
# max_features=50000
# MAX_SEQUENCE_LENGTH=303

In [ ]:
from keras import backend as K

In [ ]:
def f1(y_true, y_pred):
    '''
    metric from here 
    https://stackoverflow.com/questions/43547402/how-to-calculate-f1-macro-in-keras
    '''
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
from keras.layers import LSTM, Bidirectional, GlobalMaxPool1D, Dropout
from keras.engine.topology import Layer
from keras import initializers, regularizers, constraints

In [ ]:
# TT=np.array(FF)

In [ ]:
# # num_words = min(max_features, len(TT))
# num_words=len(TT)

In [ ]:
# from keras.preprocessing.text import Tokenizer

# t = Tokenizer()
# t.fit_on_texts(quora_best["question_text_prep"])

# vocab_size = len(t.word_index) + 1
# #vocab_size

# encoded_docs = t.texts_to_sequences(quora_best["question_text_prep"])

In [ ]:
# from numpy import zeros
# embedding_matrix = zeros((vocab_size, 300))
# for word, i in t.word_index.items():
#     embedding_vector = word_model.get(word)
#     if embedding_vector is not None:
#         embedding_matrix[i] = embedding_vector

In [ ]:
# from keras.preprocessing.sequence import pad_sequences
# max_length = 300
# padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
# # print(padded_docs)

In [ ]:
# embedding_matrix.shape

In [ ]:
# from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV


# X_train, X_val, y_train, y_val = train_test_split(padded_docs,quora_best['target'],
#                                                     train_size=0.7, random_state = 143, stratify=quora_best['target'])

In [ ]:
# model = Sequential()
# #model.add(Embedding(num_words,input_length=MAX_SEQUENCE_LENGTH,weights=[TT],trainable=False,output_dim=MAX_SEQUENCE_LENGTH))

# model.add(Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=max_length, trainable=False))
# model.add(Bidirectional(LSTM(128,dropout=0.2, recurrent_dropout=0.2, return_sequences=True))) #
# model.add(Bidirectional(LSTM(64,return_sequences=True))) # dropout=0.2, recurrent_dropout=0.2,
# #model.add(Attention(MAX_SEQUENCE_LENGTH))
# model.add(GlobalMaxPool1D())
# model.add(Dense(16, activation='relu'))
# model.add(Dropout(0.1))
# model.add(Dense(1, activation='sigmoid'))

# model.compile(loss='binary_crossentropy',
#               optimizer='adam',
#               metrics=[f1])

# print(model.summary())

In [ ]:
# batch_size = 512 #32

In [ ]:
# X_train_list=X_train.tolist()
# X_val_list=X_val.tolist()

In [ ]:
# observation_train = np.asarray(X_train_list)
# observation_val = np.asarray(X_val_list)

In [ ]:
# #weight={0:0.4,1:1}
# weight={0:1,1:2}

In [ ]:
# model.fit(X_train, y_train,
#           batch_size=batch_size,
#           epochs=2,
#           validation_data=(X_val, y_val),
#           class_weight=weight)


In [ ]:
# model.fit(observation_train, y_train,
#           batch_size=batch_size,
#           epochs=10,
#           validation_data=(observation_val, y_val),
#           class_weight=weight)

# score, f1_calc = model.evaluate(X_val, y_val,
#                             batch_size=batch_size)
# print('Test score:', score)
# print('Test f1:', f1_calc)
# #X_train, X_val, y_train, y_val

In [ ]:
# y_pred_train = model.predict(X_train)

In [ ]:
# y_pred_val = model.predict(observation_val)

In [ ]:
from tqdm.auto import tqdm

In [ ]:
# def bestThresshold(y_train,y_pred_train):
#     tmp = [0,0,0] # idx, cur, max
#     delta = 0
#     for tmp[0] in tqdm(np.arange(0.1, 0.501, 0.01)):
#         tmp[1] = f1_score(y_train, np.array(y_pred_train)>tmp[0])
#         if tmp[1] > tmp[2]:
#             delta = tmp[0]
#             tmp[2] = tmp[1]
#     print('best threshold is {:.4f} with F1 score: {:.4f}'.format(delta, tmp[2]))
#     return delta
# delta = bestThresshold(y_train,y_pred_train)

In [ ]:
# t_test = Tokenizer()
# t_test.fit_on_texts(quora_test["question_text_prep"])

# #vocab_size = len(t.word_index) + 1
# #vocab_size

# encoded_docs_test = t_test.texts_to_sequences(quora_test["question_text_prep"])

In [ ]:
# max_length = 300
# padded_docs_test = pad_sequences(encoded_docs_test, maxlen=max_length, padding='post')
# #print(padded_docs)

In [ ]:
# # test_Features = quora_test['vector'] #quora_test['joinvector_all']
# # X_test_original=test_Features.tolist()
# # observation_test = np.asarray(X_test_original)
# # y_test_pred= model.predict(observation_test)
# y_test_pred= model.predict(padded_docs_test)

In [ ]:
test_Features = quora_test['joinvector_all']
X_test_original=test_Features.tolist()
# y_test_pred= best_model.predict(X_test_original)
y_test_pred= GB_clf.predict(X_test_original)


In [ ]:
# y_test_pred

In [ ]:
quora_test_tmp=quora_test

In [ ]:
# delta=0.4

In [ ]:
quora_test_tmp["pred"]=y_test_pred #(y_test_pred > delta).astype(int) 
quora_test_tmp1 = quora_test_tmp[['qid','question_text','pred']]
quora_test_tmp1[quora_test_tmp1['pred']==1].sample(10)

In [ ]:
sub = pd.read_csv('../input/sample_submission.csv')
out_df = pd.DataFrame({"qid":sub["qid"].values})
out_df['prediction'] = y_test_pred
out_df.to_csv("submission.csv", index=False)

In [ ]:
# sub = pd.read_csv('../input/sample_submission.csv')
# out_df = pd.DataFrame({"qid":sub["qid"].values})
# out_df['prediction'] =y_test_pred# (y_test_pred > delta).astype(int) #y_test_pred
# out_df.to_csv("submission.csv", index=False)

### Let's check the proportions of the submission file

In [ ]:
round(out_df['prediction'].value_counts(normalize =True),3)*100

### We can see that the proportion of the insincire questions is similar and a little bit more than the original train file

## 8. Concluding Remarks and Work in Progress 

* ### We used  Logistic Classifier as our classification model. 
* ### We have reached the following results: f1_score: 0.8 vs. 0.79 , in the training and testing (val) sets, respectively. 

* ### The results indicate on the model's robustness as well as its ability to distiguish between insencere and sincere questions. 
### Later on we have used NN algorithems to predict the same data
### Points for further exploration:
### 1. Joining the Clustering feature to the combined vector
### 2. Trying different Classification based on Neural Net algorithems with the combined vector